# Парсер Avito 

## Подключение 

`pip install selenium`

`pip install webdriver-manager`

In [1]:
# Библиотеки для анализа данных
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

import datetime
import time

import os
import re

from tqdm import tqdm

# Библиотеки для парсинга 
import requests

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from webdriver_manager.chrome import ChromeDriverManager

# Старый парсер 
# from src.parser_avito import parse_avito_page

# Парсер 
from src.parser import parse_avito_page
from src.tools import get_ad_urls, get_photos


## Константы 

In [2]:


URL_TEMPLATE = 'https://www.avito.ru/all/avtomobili/xetcbek_5_dv-ASgBAgICAUSm_xHsh4sD?f=ASgBAgECA0SeEp64Aqb_EeyHiwPs6hSSmZADAUX6jBQXeyJmcm9tIjoyMDE1LCJ0byI6bnVsbH0&p='
PAUSE_DURATION_SECONDS = 2
NUM_PAGES = 100

columns = [
    'title', 'price', 'characteristics', 
    'description', 'location', 'link', 'photo'
]

## Запуск сервиса

In [3]:
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

## Максимальный номер сохраненной страницы  

In [4]:
output_folder = "data8"
os.makedirs(output_folder, exist_ok=True)

# Получаем список всех файлов в папке
files = os.listdir(output_folder)

# Ищем файлы с расширением .csv и извлекаем числовые части
numbers = []
for file in files:
    if file.endswith('.parquet'):
        match = re.search(r'\d+', file)
        if match:
            numbers.append(int(match.group()))

# Находим максимальное число
max_number = max(numbers) if numbers else None
print(f"Максимальный номер: {max_number}")


if not max_number: 
    start_page_number = 1
else: 
    start_page_number = max_number + 1

end_page_number = start_page_number + NUM_PAGES

print(f"Начнем парсинг со страницы номер: {start_page_number} (вкл)")
print(f"Закончим парсинг на странице номер: {end_page_number} (не вкл)")

Максимальный номер: None
Начнем парсинг со страницы номер: 1 (вкл)
Закончим парсинг на странице номер: 101 (не вкл)


## Парсинг

**Логика работы парсера:** 
1) Заходим на страницу с объявлениями
2) Собираем все ссылки на объявления
3) В цикле заходим на странички объявлений
4) Парсим текстовую информацию
5) В цикле проходимся по первым 5 фотографиям и сохраняем их ссылки 

In [ ]:
print(output_folder)

for page_num in range(start_page_number, end_page_number):
    
    print(f"Начали парсинг страницы # {page_num}")

    df_page = pd.DataFrame(columns=columns)
    
    try:
        # Загрузка страницы с объявлениями
        url = URL_TEMPLATE + str(page_num)
        driver.get(url)
        time.sleep(PAUSE_DURATION_SECONDS)

        # Получаем HTML-код страницы
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
  
        # Извлекаем ссылки на объявления на текущей странице
        links = get_ad_urls(soup)

        for link in tqdm(links):
            
            try:
                # Переход на страницу объявления
                driver.get(link)
                time.sleep(PAUSE_DURATION_SECONDS)  # Задержка для полной загрузки

                # Парсим данные на странице объявления (название, цена, фото, описание и т.д.)
                ad_data = parse_avito_page(driver=driver) # <- словарик 
                ad_data['link'] = link
                ad_data['photo'] = get_photos(driver=driver, cnt=5)

                df_page = pd.concat([df_page, pd.DataFrame([ad_data])], ignore_index=True)

                
            except TimeoutException:
                print(f"Ошибка: объявление {link} не загрузилось, пропускаем...")
                continue
            except Exception as e:
                print(f"Произошла ошибка при обработке объявления {link}: {e}")
                continue

        # Сохраняем данные для текущей страницы в DataFrame
        df_page.to_parquet(f'{output_folder}/cars_data_page_{page_num}.parquet')

    except TimeoutException:
        print(f"Ошибка: страница {page_num} не загрузилась, пропускаем...")
        continue
    except Exception as e:
        print(f"Произошла ошибка на странице {page_num}: {e}")
        continue



In [ ]:
# Закрываем драйвер после завершения
driver.quit()